In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mode
from IPython.display import display, HTML
import xlrd
import os
import glob
import xgboost as xgb
from datetime import datetime, date
from datetime import timedelta
from dateutil import tz
import json
import requests
import json
import sys

In [2]:
glob.glob("./Downloads/*.csv")

['./Downloads/Loads.csv', './Downloads/cities.csv']

In [3]:
%%time
df = pd.read_csv('./Downloads/Loads.csv', low_memory = False)

CPU times: user 950 ms, sys: 112 ms, total: 1.06 s
Wall time: 1.06 s


In [4]:
df.shape

(65535, 91)

0

In [5]:
appointmentTimeBoolMap = ~df['Appointment Time'].isna()

In [6]:
np.sum(appointmentTimeBoolMap)

65399

In [7]:
df = df.loc[appointmentTimeBoolMap, :]

In [8]:
df.shape

(65399, 91)

In [9]:
# display(df['Appointment Time'])

In [10]:
print(df.shape)

(65399, 91)


In [11]:
datetime.strptime("05/30/2019 09:45 PM", '%m/%d/%Y %I:%M %p')

datetime.datetime(2019, 5, 30, 21, 45)

In [12]:
edtTimeZone = tz.gettz('US/Eastern')
mdtTimeZone = tz.gettz('US/Mountain')
cestTimeZone = tz.gettz('Europe/Paris')
pdtTimeZone = tz.gettz('US/Pacific')
# for f in df['Appointment Time']:
def getWithTimeZone(f):
    to_zone = tz.gettz('US/Central')
    if f[-3:] == "EDT":
        edt = datetime.strptime(f[:-4], '%m/%d/%Y %I:%M %p')
        edt = edt.replace(tzinfo = edtTimeZone)
        f = edt.astimezone(to_zone)
    elif f[-3:] == "MDT":
        mdt = datetime.strptime(f[:-4], '%m/%d/%Y %I:%M %p')
        mdt = mdt.replace(tzinfo = mdtTimeZone)
        f = mdt.astimezone(to_zone)
    elif f[-4:] == "CEST":
        cest = datetime.strptime(f[:-5], '%m/%d/%Y %I:%M %p')
        cest = cest.replace(tzinfo = cestTimeZone)
        f = cest.astimezone(to_zone)
    elif f[-3:] == "PDT":
        pdt = datetime.strptime(f[:-4], '%m/%d/%Y %I:%M %p')
        pdt = pdt.replace(tzinfo = pdtTimeZone)
        f = pdt.astimezone(to_zone)
    else:
        f = datetime.strptime(f[:-4], '%m/%d/%Y %I:%M %p')
    return pd.to_datetime(f)
        
df.loc[:, 'Appointment Time'] = (df.loc[:, 'Appointment Time'].apply(lambda x: getWithTimeZone(x)))
display(df.head())

,Load Number,PRO Number,Tracking Number,Shipper,Carrier,3pl,Carrier SCAC,Customer,Stop Name,Stop Type,...,ActualDelivery-ETA 24 hours before delivery,ActualDelivery-ETA 12 hours before delivery,ActualDelivery-ETA 8 hours before delivery,ActualDelivery-ETA 4 hours before delivery,ActualDelivery-ETA 2 hours before delivery,ETA 4 hours before delivery,ETA 2 hours before delivery,ETA 4 hours before appointment,ETA 2 hours before appointment,Test Load
0,4300580413,NaN,NaN,Smithfield Foods,Stevens Transport,NaN,STVV,NaN,CU0020001852-KROGER CO-LOUISVILLE,Delivery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
1,4300580413,NaN,NaN,Smithfield Foods,Stevens Transport,NaN,STVV,NaN,"SP5087-Cloverleaf-Benson, NC",Pickup,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
2,4300579679,NaN,NaN,Smithfield Foods,Heyl Truck Lines,NaN,HEYL,NaN,CU0020000407-COSTCO 1036 AIRDRIE DRY,Delivery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
3,4300579679,NaN,NaN,Smithfield Foods,Heyl Truck Lines,NaN,HEYL,NaN,"SP1233-SFD-Monmouth,IL",Pickup,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
4,4300579366,NaN,NaN,Smithfield Foods,Heyl Truck Lines,NaN,HEYL,NaN,CU0020000407-COSTCO 1036 AIRDRIE DRY,Delivery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO


In [13]:
# print(df['Appointment Time'])

In [14]:
strtTime = pd.to_datetime("2019-05-31", format = '%Y-%m-%d').date()
endTime = pd.to_datetime("2019-06-08", format = '%Y-%m-%d').date()
df = df[df['Appointment Time'].transform(lambda x: x.date() > strtTime and x.date() < endTime)]
display(df.shape)

(159, 91)

In [7]:
stopboolmap = df.groupby('Load Number')['Stop Type'].transform(lambda x: len(x) == 2)
df = df.loc[stopboolmap, :]
display(df.shape)

(50238, 91)

In [17]:
print(df.loc[:, 'Appointment Time'].shape)

(58,)


In [18]:
display(df['Stop Type'].shape)

(58,)

In [19]:
display(df.shape)

(58, 91)

In [20]:
display(len(df['Truck Number'].unique()))

21

In [21]:
display(df)

,Load Number,PRO Number,Tracking Number,Shipper,Carrier,3pl,Carrier SCAC,Customer,Stop Name,Stop Type,...,ActualDelivery-ETA 24 hours before delivery,ActualDelivery-ETA 12 hours before delivery,ActualDelivery-ETA 8 hours before delivery,ActualDelivery-ETA 4 hours before delivery,ActualDelivery-ETA 2 hours before delivery,ETA 4 hours before delivery,ETA 2 hours before delivery,ETA 4 hours before appointment,ETA 2 hours before appointment,Test Load
39,4300575947,NaN,NaN,Smithfield Foods,Raider Express Inc.,NaN,RDXO,NaN,CU0010001351-CHICKEN E FOODSERVICE INC,Delivery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
40,4300575947,NaN,NaN,Smithfield Foods,Raider Express Inc.,NaN,RDXO,NaN,"SP1264-SFD-Cudahy, WI",Pickup,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
167,4300579119,NaN,NaN,Smithfield Foods,"UTXL, Inc.",NaN,UNFF,NaN,CU0020019609-LOCKE COLD STORAGE CHICAGO IL,Delivery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
168,4300579119,NaN,NaN,Smithfield Foods,"UTXL, Inc.",NaN,UNFF,NaN,"SP5020-Des Moines CS Court , IA",Pickup,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
205,4300577835,NaN,NaN,Smithfield Foods,Schuster Co.,NaN,SHGR,NaN,"SP1264-SFD-Cudahy, WI",Delivery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
206,4300577835,NaN,NaN,Smithfield Foods,Schuster Co.,NaN,SHGR,NaN,"SP1265-SFD-Sioux Center, IA",Pickup,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
207,4300576516,NaN,NaN,Smithfield Foods,Hirschbach Motor Lines Inc,NaN,HIRS,NaN,CFS-ITLJOLIET-CFS-ITL JOLIET,Delivery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
208,4300576516,NaN,NaN,Smithfield Foods,Hirschbach Motor Lines Inc,NaN,HIRS,NaN,"SP5149-Cloverleaf-Chillicothe,MO",Pickup,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
231,4300580087,NaN,NaN,Smithfield Foods,R.E. Garrison Logistics,NaN,GRLB,NaN,CU0020001648-WAL-MART DC 6084,Delivery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
232,4300580087,NaN,NaN,Smithfield Foods,R.E. Garrison Logistics,NaN,GRLB,NaN,"SP5310-Noram - Le Mars,IA",Pickup,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO


In [22]:
print(df['Carrier'])

39                 Raider Express Inc.
40                 Raider Express Inc.
167                         UTXL, Inc.
168                         UTXL, Inc.
205                       Schuster Co.
206                       Schuster Co.
207         Hirschbach Motor Lines Inc
208         Hirschbach Motor Lines Inc
231            R.E. Garrison Logistics
232            R.E. Garrison Logistics
239          Pro Transport and Leasing
240          Pro Transport and Leasing
265                       Schuster Co.
266                       Schuster Co.
277        Corney Transportation, Inc.
278        Corney Transportation, Inc.
345                         Prime Inc.
346                         Prime Inc.
492                  Stevens Transport
493                  Stevens Transport
764                         Prime Inc.
765                         Prime Inc.
978                       Schuster Co.
979                       Schuster Co.
982                       Schuster Co.
983                      

In [23]:
df.loc[(df['Carrier'] != "Schuster Co."), 'Truck Number'] = np.nan

In [24]:
df.loc[(df['Carrier'] != "Schuster Co."), 'Carrier'] = np.nan

In [25]:
display(df.head())

,Load Number,PRO Number,Tracking Number,Shipper,Carrier,3pl,Carrier SCAC,Customer,Stop Name,Stop Type,...,ActualDelivery-ETA 24 hours before delivery,ActualDelivery-ETA 12 hours before delivery,ActualDelivery-ETA 8 hours before delivery,ActualDelivery-ETA 4 hours before delivery,ActualDelivery-ETA 2 hours before delivery,ETA 4 hours before delivery,ETA 2 hours before delivery,ETA 4 hours before appointment,ETA 2 hours before appointment,Test Load
39,4300575947,NaN,NaN,Smithfield Foods,NaN,NaN,RDXO,NaN,CU0010001351-CHICKEN E FOODSERVICE INC,Delivery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
40,4300575947,NaN,NaN,Smithfield Foods,NaN,NaN,RDXO,NaN,"SP1264-SFD-Cudahy, WI",Pickup,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
167,4300579119,NaN,NaN,Smithfield Foods,NaN,NaN,UNFF,NaN,CU0020019609-LOCKE COLD STORAGE CHICAGO IL,Delivery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
168,4300579119,NaN,NaN,Smithfield Foods,NaN,NaN,UNFF,NaN,"SP5020-Des Moines CS Court , IA",Pickup,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
205,4300577835,NaN,NaN,Smithfield Foods,Schuster Co.,NaN,SHGR,NaN,"SP1264-SFD-Cudahy, WI",Delivery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO


In [26]:
display(df['Truck Number'])

39          NaN
40          NaN
167         NaN
168         NaN
205      331288
206      331288
207         NaN
208         NaN
231         NaN
232         NaN
239         NaN
240         NaN
265      780323
266      780323
277         NaN
278         NaN
345         NaN
346         NaN
492         NaN
493         NaN
764         NaN
765         NaN
978         NaN
979         NaN
982         NaN
983         NaN
1058       3062
1059       3062
1450        NaN
1451        NaN
1452        NaN
1453        NaN
1755        NaN
1756        NaN
1757        NaN
1758        NaN
2515        NaN
2516        NaN
2540        NaN
2541        NaN
2593        NaN
2594        NaN
2595        NaN
2596        NaN
3325        NaN
3326        NaN
5281        NaN
5282        NaN
5572        NaN
5573        NaN
5809        NaN
5810        NaN
5811        NaN
5812        NaN
5817        NaN
5818        NaN
21223       NaN
21224       NaN
Name: Truck Number, dtype: object

In [27]:
df['City']

39             Burleson
40               Cudahy
167             Chicago
168          Des Moines
205              Cudahy
206        Sioux Center
207              Joliet
208         Chillicothe
231           Los Lunas
232             Le Mars
239             Alachua
240            Tar Heel
265              Temple
266             Le Mars
277             Alachua
278            Tar Heel
345             Corinne
346               Crete
492             Houston
493              Temple
764           Grandview
765               Crete
978              London
979             Le Mars
982           New Caney
983             Le Mars
1058       Indianapolis
1059         Greenfield
1450             Laredo
1451           Tar Heel
1452             Laredo
1453           Tar Heel
1755         Petersburg
1756        Orange City
1757         Petersburg
1758        Orange City
2515            Arcadia
2516              Cocoa
2540          Los Lunas
2541       Pauls Valley
2593       Gordonsville
2594            

In [28]:
for f in df['City']:
    s = str(f).lower()
    df.loc[df['City'] == f, 'City'] = s

In [29]:
df.loc[:, 'City'] = df.loc[:, 'City'] + ',' + df.loc[:, 'State']

In [30]:
display(df['City'])

39             burleson,TX
40               cudahy,WI
167             chicago,IL
168          des moines,IA
205              cudahy,WI
206        sioux center,IA
207              joliet,IL
208         chillicothe,MO
231           los lunas,NM
232             le mars,IA
239             alachua,FL
240            tar heel,NC
265              temple,TX
266             le mars,IA
277             alachua,FL
278            tar heel,NC
345             corinne,UT
346               crete,NE
492             houston,TX
493              temple,TX
764           grandview,WA
765               crete,NE
978              london,KY
979             le mars,IA
982           new caney,TX
983             le mars,IA
1058       indianapolis,IN
1059         greenfield,IN
1450             laredo,TX
1451           tar heel,NC
1452             laredo,TX
1453           tar heel,NC
1755         petersburg,VA
1756        orange city,IA
1757         petersburg,VA
1758        orange city,IA
2515            arcadia,FL
2

In [31]:

df2 = pd.read_csv('./Downloads/cities.csv')
df2.drop_duplicates(subset = ['city'], keep = 'last', inplace = True)

class Pickup:
    def __init__(self):
        pass
class Delivery:
    def __init__(self):
        pass

class Load:
    def __init__(self):
        self.pickup = Pickup()
        self.delivery = Delivery()

class Initial_Position:
    def __init__(self):
        pass
class Truck:
    def __init__(self):
        self.Initial_Position = Initial_Position()
L = []

for f, o in (df.groupby('Load Number')['City']):
    o = o.values
    load = Load()
    L.append(load)
    load.Id = f
    load.pickup.Latitude = df2.loc[(df2['city'] == o[1]), 'latitude'].values[0]
    load.pickup.Longitude = df2.loc[(df2['city'] == o[1]), 'longitude'].values[0]
    load.delivery.Latitude = df2.loc[(df2['city'] == o[0]), 'latitude'].values[0]
    load.delivery.Longitude = df2.loc[(df2['city'] == o[0]), 'longitude'].values[0]
    StrtDate = (df.loc[(df['City'] == o[1]) & (df['Load Number'] == load.Id), 'Appointment Time'].dt.date)
    FinDate = df.loc[(df['City'] == o[0]) & (df['Load Number'] == load.Id), 'Appointment Time'].dt.date
    StrtDateTime = pd.to_datetime(StrtDate, format = '%Y-%m-%d %H:%M:%S')
    FinDateTime = pd.to_datetime(FinDate, format = '%Y-%m-%d %H:%M:%S')
    load.pickup.EarlyAppointmentTime = str(StrtDateTime.values[0]).split('T')
    load.pickup.EarlyAppointmentTime = str(load.pickup.EarlyAppointmentTime[0])+' '+str((str(load.pickup.EarlyAppointmentTime[1]).split('.'))[0])
    load.pickup.EarlyAppointmentTime = datetime.strptime(load.pickup.EarlyAppointmentTime[:18], '%Y-%m-%d %H:%M:%S')
    #print(load.pickup.EarlyAppointmentTime)
    load.delivery.EarlyAppointmentTime = str(FinDateTime.values[0]).split('T')
    load.delivery.EarlyAppointmentTime = str(load.delivery.EarlyAppointmentTime[0])+' '+str((str(load.delivery.EarlyAppointmentTime[1]).split('.'))[0])
    load.delivery.EarlyAppointmentTime = datetime.strptime(load.delivery.EarlyAppointmentTime[:18], '%Y-%m-%d %H:%M:%S')
    #print(load.pickup.EarlyAppointmentTime, load.delivery.EarlyAppointmentTime)
    load.pickup.LatestAppointmentTime = load.pickup.EarlyAppointmentTime + timedelta(hours = 23, minutes = 59, seconds = 59) 
    load.delivery.LatestAppointmentTime = load.delivery.EarlyAppointmentTime + timedelta(hours = 23, minutes = 59, seconds = 59)
    #print(load.pickup.LatestAppointmentTime, load.delivery.LatestAppointmentTime)
    #print(load.pickup.EarlyAppointmentTime, load.pickup.LatestAppointmentTime, load.delivery.EarlyAppointmentTime, load.delivery.LatestAppointmentTime)
#     break


In [32]:
for l in L:
    print(l.pickup.Latitude)

36.456390380859375
36.456390380859375
35.72126770019531
42.959739685058594
36.456390380859375
39.79529571533203
40.62521743774414
43.07979965209961
40.62521743774414
43.00674057006836
35.47987747192383
35.47987747192383
32.91100311279297
42.79415512084961
39.78504180908203
41.58683395385742
39.739234924316406
43.00720977783203
39.739234924316406
39.739234924316406
36.456390380859375
31.098234176635746
34.74008178710937
42.79415512084961
42.79415512084961
42.79415512084961
28.32000732421875
43.00720977783203
43.00720977783203


In [33]:
for load in L:
    url = "https://route.api.here.com/routing/7.2/calculateroute.json?\
app_id=SWBYU0csWZrpTY9aQBqL&app_code=GlpK2MA4IGFM2MsPX3ICPg&gen=9&mode=\
fastest%3Btruck%3Btraffic%3Aenabled&legattributes=none%2Cle%2Ctt&\
resolution=10&routeattributes=none%2Clg&departure=2019-06-04T21%3A38%\
3A02%2B00%3A00&waypoint0=geo%21{}%2C{}&waypoint1=geo%21{}%2C{}".format(
        load.pickup.Latitude, load.pickup.Longitude, load.delivery.Latitude, load.delivery.Longitude)
    print(load.pickup.Latitude, load.pickup.Longitude, load.delivery.Latitude, load.delivery.Longitude)
    info = requests.get(url).json()['response']['route'][0]['leg'][0]
    load.Distance = info['length']
    load.TimeTaken = info['travelTime']

36.456390380859375 -76.8566665649414 27.50356101989746 -99.50755310058594
36.456390380859375 -76.8566665649414 27.50356101989746 -99.50755310058594
35.72126770019531 -77.91554260253906 36.98237228393555 -76.63106536865234
42.959739685058594 -87.8614730834961 32.54208374023437 -97.3208465576172
36.456390380859375 -76.8566665649414 29.793813705444336 -82.49442291259766
39.79529571533203 -93.55243682861328 41.52503204345703 -88.08172607421875
40.62521743774414 -96.96151733398438 46.2509651184082 -119.90170288085938
43.07979965209961 -96.17568969726562 42.959739685058594 -87.8614730834961
40.62521743774414 -96.96151733398438 41.55104446411133 -112.1102294921875
43.00674057006836 -74.36763763427734 44.10034942626953 -70.21477508544922
35.47987747192383 -79.1802978515625 31.808767318725586 -85.9699478149414
35.47987747192383 -79.1802978515625 38.13735580444336 -78.18778228759766
32.91100311279297 -111.7733917236328 33.980602264404304 -117.37549591064452
42.79415512084961 -96.16558074951172 3

In [34]:
for l in L:
    print((l.Distance), (l.TimeTaken))

2739880 112061
2739880 112061
248395 12607
1749108 69587
1103440 44830
619673 25388
2404389 89844
816803 33451
1446107 53746
558512 24540
942095 40634
442839 18815
590275 25528
1444504 58632
40509 2536
537278 22202
1104206 47691
645152 26803
1104206 47691
1104206 47691
1103440 44830
297920 14124
1003119 38754
1819354 67849
1630962 65711
1547061 61861
266094 14626
2221771 91974
2221771 91974


In [35]:
for f in L:
    s = datetime.fromtimestamp(f.TimeTaken)
    l = datetime.fromtimestamp(0)
    f.TimeTaken = (s-l)*21.0/11

In [36]:
#Id, Inital_Position, Final_Position, Possible, PickupTime, EAT
T = []
for f, o in df.groupby('Truck Number')['City']:
    truck = Truck()
    T.append(truck)
    if f != np.nan:
        o = o.values
        truck.Id = f
        truck.Initial_Position.Latitude = df2.loc[(df2['city'] == o[1]), 'latitude'].values[0]
        truck.Initial_Position.Longitude = df2.loc[(df2['city'] == o[1]), 'longitude'].values[0]
        truck.PickupTime = df.loc[(df['Truck Number'] == truck.Id) & (df['City'] == o[1]), 'Appointment Time'].values[0]
        truck.PickupTime = datetime.strptime(str(truck.PickupTime)[:18], '%Y-%m-%d %H:%M:%S')
        print(truck.PickupTime)


2019-06-03 08:30:00
2019-06-01 22:30:00
2019-06-01 08:30:00


In [ ]:
%%time
# Preference, Distance, EAT, Possible, IndexNumber
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None
        self.prev = None
def Preference_Truck_Load(a, b):
    if a > b:
        return 1
    elif a < b and a >= b - timedelta(days = 1):
        return 2
    elif a < b - timedelta(days = 1) and a >= b - timedelta(days = 2):
        return 3
    elif a < b - timedelta(days = 2) and a >= b - timedelta(days = 3):
        return 4
    elif a < b - timedelta(days = 3) and a >= b - timedelta(days = 4):
        return 5
    elif a < b - timedelta(days = 4) and a >= b - timedelta(days = 5):
        return 6
    else:
        return 7
class Truck_Load_Assignment:
    def __init__(self):
        pass
    
    def __lt__(self, other):
#         print("Called")
        if(self.Preference != other.Preference):
            return self.Preference < other.Preference
        return self.Dead_Head > other.Dead_Head
TLA = [[] for _ in range(len(T))]
for t in range(len(T)):
    Index = 1
    for l in L:
        truck_load_assignment = Truck_Load_Assignment()
        url = "https://route.api.here.com/routing/7.2/calculateroute.json?\
app_id=SWBYU0csWZrpTY9aQBqL&app_code=GlpK2MA4IGFM2MsPX3ICPg&gen=9&mode=\
fastest%3Btruck%3Btraffic%3Aenabled&legattributes=none%2Cle%2Ctt&\
resolution=10&routeattributes=none%2Clg&departure=2019-06-04T21%3A38%\
3A02%2B00%3A00&waypoint0=geo%21{}%2C{}&waypoint1=geo%21{}%2C{}".format(
            T[t].Initial_Position.Latitude, T[t].Initial_Position.Longitude, l.pickup.Latitude, l.pickup.Longitude)
        apiReturn = requests.get(url).json()
        info = apiReturn['response']['route'][0]['leg'][0]
        distance = info['length']
        time = info['travelTime']
        s = datetime.fromtimestamp(time)
        l1 = datetime.fromtimestamp(0)
        s = s-l1
        s = T[t].PickupTime + s*21.0/11
        if s < l.pickup.LatestAppointmentTime:
            truck_load_assignment.Possible = True
            truck_load_assignment.Preference = Preference_Truck_Load(s, l.pickup.EarlyAppointmentTime)
            truck_load_assignment.Dead_Head = distance
            truck_load_assignment.E_Pickup_T = s
            truck_load_assignment.E_Delivery_T = s+l.TimeTaken
            truck_load_assignment.TimeTaken = l.TimeTaken
            truck_load_assignment.Index = Index
            truck_load_assignment.No_of_Loads = 1
            truck_load_assignment.head = Node(Index)
            truck_load_assignment.other = Node(0)
            truck_load_assignment.tother = truck_load_assignment.other
            truck_load_assignment.tail = truck_load_assignment.head
            truck_load_assignment.Final_Latitude = l.delivery.Latitude
            truck_load_assignment.Final_Longitude = l.delivery.Longitude
            truck_load_assignment.Initial_Latitude = l.pickup.Latitude
            truck_load_assignment.Initial_Longitude = l.pickup.Longitude
            truck_load_assignment.LatestAppointmentTime = l.delivery.LatestAppointmentTime
            truck_load_assignment.EarlyAppointmentTime = l.delivery.EarlyAppointmentTime
            truck_load_assignment.Previous_In = -1
        else:
            truck_load_assignment.Possible = False
            truck_load_assignment.Index = Index
            truck_load_assignment.head = Node(Index)
            truck_load_assignment.tail = truck_load_assignment.head
            truck_load_assignment.No_of_Loads = 0
            truck_load_assignment.Preference = 8
            truck_load_assignment.Previous_In = -1
            truck_load_assignment.other = Node(0)
            truck_load_assignment.tother = truck_load_assignment.other
            truck_load_assignment.Dead_Head = sys.maxsize
        TLA[t].append(truck_load_assignment)
        Index = Index + 1
    TLA[t] = sorted(TLA[t])
for i in range(len(T)):
    for j in range(len(L)):
        print(TLA[i][j].Preference, TLA[i][j].Dead_Head)

Travel_T = [[] for _ in range(len(L))]
Dead = [[] for _ in range(len(L))]
for i in range(len(L)):
    for j in range(len(L)):
        if(i == j):
            Dead[i].append(0)
            Travel_T.append(0)
        else:
            url = "https://route.api.here.com/routing/7.2/calculateroute.json?\
app_id=SWBYU0csWZrpTY9aQBqL&app_code=GlpK2MA4IGFM2MsPX3ICPg&gen=9&mode=\
fastest%3Btruck%3Btraffic%3Aenabled&legattributes=none%2Cle%2Ctt&\
resolution=10&routeattributes=none%2Clg&departure=2019-06-04T21%3A38%\
3A02%2B00%3A00&waypoint0=geo%21{}%2C{}&waypoint1=geo%21{}%2C{}".format(
                 L[i].delivery.Latitude, L[i].delivery.Longitude, L[j].pickup.Latitude, L[j].pickup.Longitude)
            api_return = requests.get(url).json()
            info = api_return['response']['route'][0]['leg'][0]
            Dead[i].append(info['length'])
            s = datetime.fromtimestamp(info['travelTime'])
            l1 = datetime.fromtimestamp(0)
            s = (s-l1)*21.0/11
            Travel_T[i].append(s)
        

1 480809
1 0
2 1788835
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
1 1208014
1 1051873
1 549666
1 391540
1 32505
1 32505
1 32505
1 16
2 816803
2 32505
2 17892
3 17892
3 17892
4 1093076
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8 9223372036854775807
8

In [ ]:
# r = TLA[0][2].head
# while(r != None):
#     print(r.data)
#     r = r.next

In [ ]:
%%time
def MAX(a, b):
    if a > b:
        return a
    return b

for i in range(len(T)):
    for j in range(1, len(L)):
        x = TLA[i][j].No_of_Loads
        Dead_Head = 0
        E_Pickup_T = datetime.fromtimestamp(0)
        E_Delivery_T = datetime.fromtimestamp(0)
        if(TLA[i][j].Preference < 8):
            In = -1
            for k in range(j):
                if TLA[i][k].Preference < TLA[i][j].Preference:
                    y = TLA[i][j].No_of_Loads
                    if(TLA[i][k].E_Delivery_T + Travel_T[k][j] + TLA[i][j].TimeTaken <= TLA[i][j].LatestAppointmentTime):
                        l = MAX(x, TLA[i][k].No_of_Loads + y)
                        if(TLA[i][k].No_of_Loads + y == x):
                            if(Dead[k][j] + TLA[i][k].Dead_Head < Dead_Head):
                                Dead_Head = TLA[i][k].Dead_Head + Dead[k][j]
                                In = k
                                E_Pickup_T = TLA[i][k].E_Delivery_T + Travel_T[k][j]
                                E_Delivery_T = E_Pickup_T + TLA[i][j].TimeTaken
                        elif(TLA[i][k].No_of_Loads + y > x):
                            Dead_Head = Dead[k][j] + TLA[i][k].Dead_Head
                            In = k
                            E_Pickup_T = TLA[i][k].E_Delivery_T + Travel_T[k][j]
                            E_Delivery_T = E_Pickup_T + TLA[i][j].TimeTaken
                        x = l
            if In > -1:
                q = TLA[i][In].head
                while(q != None):
                    TLA[i][j].tail.next = Node(q.data)
                    TLA[i][j].tail.next.prev = TLA[i][j].tail
                    TLA[i][j].tail = TLA[i][j].tail.next
                    q = q.next
                TLA[i][j].Dead_Head = Dead_Head
                TLA[i][j].E_Pickup_T = E_Pickup_T
                TLA[i][j].E_Delivery_T = E_Delivery_T
        TLA[i][j].No_of_Loads = x


                
TD = [[] for _ in range(len(L))]
for i in range(len(T)):
    for j in range(len(L)):
        url = "https://route.api.here.com/routing/7.2/calculateroute.json?\
app_id=SWBYU0csWZrpTY9aQBqL&app_code=GlpK2MA4IGFM2MsPX3ICPg&gen=9&mode=\
fastest%3Btruck%3Btraffic%3Aenabled&legattributes=none%2Cle%2Ctt&\
resolution=10&routeattributes=none%2Clg&departure=2019-06-04T21%3A38%\
3A02%2B00%3A00&waypoint0=geo%21{}%2C{}&waypoint1=geo%21{}%2C{}".format(
                 T[i].Initial_Position.Latitude, T[i].Initial_Position.Longitude, L[j].pickup.Latitude, L[j].pickup.Longitude)
        api_return = requests.get(url).json()
        info = api_return['response']['route'][0]['leg'][0]
        TD[i].append(info['length'])            
        

In [ ]:
for i in range(len(T)):
    for j in range(len(L)):
        if(TLA[i][j].Preference < 8):
            current = TLA[i][j].head
            while(current != None):
                print(current.data)
                current = current.next
            print("------__________------- ")
    print(" ")

In [ ]:
for i in range(len(T)):
    for j in range(len(L)):
        if(TLA[i][j].Preference < 8):
            temp = None
            current = TLA[i][j].head
            while(current != None):
                temp = current.prev
                current.prev = current.next
                current.next = temp
                current = current.prev
            if(temp != None):
                TLA[i][j].head = temp.prev
            current = TLA[i][j].head
            while(current != None):
                print(current.data)
                current = current.next
            print("------__________------- ")
    print(" ")
        
            

In [ ]:
for i in range(1, 2):
    Distance = [[] for _ in range(len(L))]
    for j in range(len(L)):
        for k in range(len(L)):
            Distance[j].append(TLA[i][j].Dead_Head+TLA[i-1][k].Dead_Head)
for i in range(1, 2):
    for j in range(len(L)):
        if(TLA[i][j].Preference < 8):
            sum = 0
            x = TLA[i][j].No_of_Loads
            Dead_Head = TLA[i][j].Dead_Head
            Previous_In = -1
            for k in range(len(L)):
                if(TLA[i][k].Preference < 8):
                    y = TLA[i-1][k].No_of_Loads
                    p = TLA[i][j].head
                    while(p != None):
                        q = TLA[i-1][k].head
                        while(q != None):
                            if(p.data == q.data):
                                if(p == TLA[i][j].head):
                                    if(p.next == None):
                                        if(q == TLA[i-1][k].head):
                                            if(q.next == None):
                                                if(TLA[i-1][k].Dead_Head < TLA[i][j].Dead_Head):
                                                    Distance[j][k] -= TLA[i][j].Dead_Head
                                                else:
                                                    Distance[j][k] -= TLA[i-1][k].Dead_Head
                                            else:
                                                if(TD[i-1][q.next.data-1]+TLA[i][j].Dead_Head < TD[i-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                    Distance[j][k] += (TD[i-1][q.next.data-1]-TD[i-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                else:
                                                    Distance[j][k] -= TLA[i][j].Dead_Head
                                        else:
                                            if(q.next == None):
                                                if(Dead[q.prev.data-1][q.data-1] < TLA[i][j].Dead_Head):
                                                    Distance[j][k] -= TLA[i][j].Dead_Head
                                                else:
                                                    Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                            else:
                                                if(Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1] < Dead[q.prev.data-1][q.next.data-1]+TLA[i][j].Dead_Head):
                                                    Distance[j][k] -= TLA[i][j].Dead_Head
                                                else:
                                                    Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                    else:
                                        if(q == TLA[i-1][k].head):
                                            if(q.next == None):
                                                if(TD[i][p.next.data-1]+TLA[i-1][k].Dead_Head < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                else:
                                                    Distance[j][k] -= TLA[i-1][k].Dead_Head
                                            else:
                                                if(TD[i-1][q.next.data-1] < TD[i][p.next.data-1]):
                                                    Distance[j][k] += (TD[i-1][q.next.data-1]-TD[i-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                else:
                                                    Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                        else:
                                            if(q.next == None):
                                                if(TD[i][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                else:
                                                    Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                            else:
                                                if(Dead[q.prev.data-1][q.next.data-1] < TD[i][p.next.data-1]):
                                                    Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data]-Dead[q.data-1][q.next.data-1])
                                                else:
                                                    Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                else:
                                    if(p.next == None):
                                        if(q == TLA[i-1][k].head):
                                            if(q.next == None):
                                                if(Dead[p.prev.data-1][p.data-1] < TLA[i-1][k].Dead_Head):
                                                    Distance[j][k] -= TLA[i-1][k].Dead_Head
                                                else:
                                                    Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                            else:
                                                if(TD[i-1][q.data-1]+Dead[q.data-1][q.next.data-1] < Dead[p.prev.data-1][p.data-1]+TD[i-1][q.next.data-1]):
                                                    Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                                else:
                                                    Distance[j][k] += (TD[i-1][q.next.data-1]-TD[i-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                        else:
                                            if(q.next == None):
                                                if(Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]):
                                                    Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                                else:
                                                    Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                            else:
                                                if(Dead[p.prev.data-1][p.data-1]+Dead[q.prev.data-1][q.next.data-1] < Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                    Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                else:
                                                    Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                    else:
                                        if(q == TLA[i-1][k].head):
                                            if(q.next == None):
                                                if(Dead[p.prev.data-1][p.next.data-1] + TLA[i-1][k].Dead_Head < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                else:
                                                    Distance[j][k] -= TLA[i-1][k].Dead_Head
                                            else:
                                                if(TD[i-1][q.next.data-1] < Dead[p.prev.data-1][p.next.data-1]):
                                                    Distance[j][k] += (TD[i-1][q.next.data-1]-TD[i-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                else:
                                                    Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                        else:
                                            if(q.next == None):
                                                if(Dead[p.prev.data-1][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                else:
                                                    Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                            else:
                                                if(Dead[p.prev.data-1][p.next.data-1] < Dead[q.prev.data-1][q.next.data-1]):
                                                    Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                else:
                                                    Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                sum += 1
                                break
                            q = q.next
                        p = p.next
                    l = MAX(x, TLA[i][j].No_of_Loads+y-sum)
                    if(TLA[i][j].No_of_Loads+y-sum == x):
                        if(Dead_Head > Distance[j][k]):
                            Previous_In = k
                            Dead_Head = Distance[j][k]
                    elif(TLA[i][j].No_of_Loads+y-sum > x):
                        Previous_In = k
                        Dead_Head = Distance[j][k]
                    x = l
            if(sum > 0):
                p = TLA[i][j].head
                while(p != None):
                    q = TLA[i-1][Previous_In].head
                    while(q != None):
                        if(p.data == q.data):
                            if(p == TLA[i][j].head):
                                if(p.next == None):
                                    if(q == TLA[i-1][Previous_In].head):
                                        if(q.next == None):
                                            if(TLA[i][j].Dead_Head > TLA[i-1][Previous_In].Dead_Head):
                                                TLA[i][j].head = None
                                        else:
                                            if(TD[i-1][q.data-1]+Dead[q.data-1][q.next.data-1] < TLA[i][j].Dead_Head+TD[i-1][q.next.data-1]):
                                                TLA[i][j].head = None
                                    else:
                                        if(q.next == None):
                                            if(Dead[q.prev.data-1][q.data-1] < TLA[i][j].Dead_Head):
                                                TLA[i][j].head = None
                                        else:
                                            if(TLA[i][j].Dead_Head+Dead[q.prev.data-1][q.next.data-1] > Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                TLA[i][j].head = None
                                else:
                                    if(q == TLA[i-1][Previous_In].head):
                                        if(q.next == None):
                                            if(TD[i][p.next.data-1] + TLA[i-1][Previous_In].Dead_Head < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                TLA[i][j].head = p.next
                                                p.next.prev = None
                                                p.next = None
                                                p.prev = None
                                        else:
                                            if(TD[i-1][q.next.data-1] > TD[i][p.next.data-1]):
                                                TLA[i][j].head = p.next
                                                p.next.prev = None
                                                p.prev = None
                                                p.next = None
                                    else:
                                        if(q.next == None):
                                            if(TD[i][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                TLA[i][j].head = p.next
                                                p.next.prev = None
                                                p.prev = None
                                                p.next = None
                                        else:
                                            if(TD[i][p.next.data-1] < Dead[q.prev.data-1][q.next.data-1]):
                                                TLA[i][j].head = p.next
                                                p.next.prev = None
                                                p.prev = None
                                                p.next = None
                            else:
                                if(p.next == None):
                                    if(q == TLA[i-1][Previous_In].head):
                                        if(q.next == None):
                                            if(Dead[p.prev.data-1][p.data-1] > TLA[i-1][Previous_In].Dead_Head):
                                                p.prev.next = None
                                                p.prev = None
                                        else:
                                            if(TD[i-1][q.data-1]+Dead[q.data-1][q.next.data-1] < Dead[p.prev.data-1][p.data-1]+TD[i-1][q.next.data-1]):
                                                p.prev.next = None
                                                p.prev = None
                                    else:
                                        if(q.next == None):
                                            if(Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]):
                                                p.prev.next = None
                                                p.prev = None
                                        else:
                                            if(Dead[p.prev.data-1][p.data-1]+Dead[q.prev.data-1][q.next.data-1] > Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                p.prev.next = None
                                                p.prev = None
                                else:
                                    if(q == TLA[i-1][Previous_In].head):
                                        if(q.next == None):
                                            if(Dead[p.prev.data-1][p.next.data-1] + TLA[i-1][Previous_In].Dead_Head < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                p.prev.next = p.next
                                                p.next.prev = p.prev
                                                p.next = None
                                                p.prev = None
                                        else:
                                            if(TD[i-1][q.next.data-1] > Dead[p.prev.data-1][p.next.data-1]):
                                                p.prev.next = p.next
                                                p.next.prev = p.prev
                                                p.prev = None
                                                p.next = None
                                    else:
                                        if(q.next == None):
                                            if(Dead[p.prev.data-1][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                p.prev.next = p.next
                                                p.next.prev = p.prev
                                                p.prev = None
                                                p.next = None
                                        else:
                                            if(Dead[p.prev.data-1][p.next.data-1] < Dead[q.prev.data-1][q.next.data-1]):
                                                p.prev.next = p.next
                                                p.next.prev = p.prev
                                                p.prev = None
                                                p.next = None
                            break
                        q = q.next
                    p = p.next
            q = TLA[i-1][Previous_In].head
            while(q != None):
                p = TLA[i][j].head
                o = 1
                while(p != None):
                    if(p.data == q.data):
                        o = 0
                        break
                    p = p.next
                if(o == 1):
                    if(TLA[i][j].other.data == 0):
                        TLA[i][j].other = Node(q.data)
                        TLA[i][j].tother = TLA[i][j].other
                    else:
                        TLA[i][j].tother.next = Node(q.data)
                        TLA[i][j].tother.next.prev = TLA[i][j].tother
                        TLA[i][j].tother = TLA[i][j].tother.next
                q = q.next
            TLA[i][j].No_of_Loads = x
            TLA[i][j].Dead_Head = Dead_Head
            TLA[i][j].Previous_In = Previous_In
        print(TLA[i][j].Preference, TLA[i][j].No_of_Loads, TLA[i][j].Dead_Head, TLA[i][j].Previous_In)


In [ ]:
# r = TLA[1][0].head
# while(r != None):
#     print(r.data)
#     r = r.next

In [ ]:
for i in range(2, len(T)):
    for j in range(len(L)):
        Distance = [[] for _ in range(len(L))]
        if(TLA[i][j].Preference < 8):
            sum = 0
            x = TLA[i][j].No_of_Loads
            Dead_Head = TLA[i][j].Dead_Head
            Previous_In = -1
            for k in range(len(L)):
                Distance[j].append(TLA[i][j].Dead_Head+TLA[i-1][k].Dead_Head)
                if(TLA[i][k].Preference < 8):
                    p = TLA[i][j].head
                    y = TLA[i-1][k].No_of_Loads
                    while(p != None):
                        q = TLA[i-1][k].head
                        while(q != None):
                            if(p.data == q.data):
                                if(p == TLA[i][j].head):
                                    if(p.next == None):
                                        if(q == TLA[i-1][k].head):
                                            if(q.next == None):
                                                if(TD[i-1][q.data-1] < TLA[i][j].Dead_Head):
                                                    Distance[j][k] -= TLA[i][j].Dead_Head
                                                else:
                                                    Distance[j][k] -= TD[i-1][q.data-1]
                                            else:
                                                if(TD[i-1][q.next.data-1]+TLA[i][j].Dead_Head < TD[i-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                    Distance[j][k] += (TD[i-1][q.next.data-1]-TD[i-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                else:
                                                    Distance[j][k] -= TLA[i][j].Dead_Head
                                        else:
                                            if(q.next == None):
                                                if(Dead[q.prev.data-1][q.data-1] < TLA[i][j].Dead_Head):
                                                    Distance[j][k] -= TLA[i][j].Dead_Head
                                                else:
                                                    Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                            else:
                                                if(Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1] < Dead[q.prev.data-1][q.next.data-1]+TLA[i][j].Dead_Head):
                                                    Distance[j][k] -= TLA[i][j].Dead_Head
                                                else:
                                                    Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                    else:
                                        if(q == TLA[i-1][k].head):
                                            if(q.next == None):
                                                if(TD[i][p.next.data-1]+TD[i-1][q.data-1] < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                else:
                                                    Distance[j][k] -= TD[i-1][q.data-1]
                                            else:
                                                if(TD[i-1][q.next.data-1] < TD[i][p.next.data-1]):
                                                    Distance[j][k] += (TD[i-1][q.next.data-1]-TD[i-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                else:
                                                    Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                        else:
                                            if(q.next == None):
                                                if(TD[i][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                else:
                                                    Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                            else:
                                                if(Dead[q.prev.data-1][q.next.data-1] < TD[i][p.next.data-1]):
                                                    Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data]-Dead[q.data-1][q.next.data-1])
                                                else:
                                                    Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                else:
                                    if(p.next == None):
                                        if(q == TLA[i-1][k].head):
                                            if(q.next == None):
                                                if(Dead[p.prev.data-1][p.data-1] < TD[i-1][q.data-1]):
                                                    Distance[j][k] -= TD[i-1][q.data-1]
                                                else:
                                                    Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                            else:
                                                if(TD[i-1][q.data-1]+Dead[q.data-1][q.next.data-1] < Dead[p.prev.data-1][p.data-1]+TD[i-1][q.next.data-1]):
                                                    Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                                else:
                                                    Distance[j][k] += (TD[i-1][q.next.data-1]-TD[i-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                        else:
                                            if(q.next == None):
                                                if(Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]):
                                                    Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                                else:
                                                    Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                            else:
                                                if(Dead[p.prev.data-1][p.data-1]+Dead[q.prev.data-1][q.next.data-1] < Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                    Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                else:
                                                    Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                    else:
                                        if(q == TLA[i-1][k].head):
                                            if(q.next == None):
                                                if(Dead[p.prev.data-1][p.next.data-1] + TD[i-1][q.data-1] < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                else:
                                                    Distance[j][k] -= TD[i-1][q.data-1]
                                            else:
                                                if(TD[i-1][q.next.data-1] < Dead[p.prev.data-1][p.next.data-1]):
                                                    Distance[j][k] += (TD[i-1][q.next.data-1]-TD[i-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                else:
                                                    Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                        else:
                                            if(q.next == None):
                                                if(Dead[p.prev.data-1][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                else:
                                                    Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                            else:
                                                if(Dead[p.prev.data-1][p.next.data-1] < Dead[q.prev.data-1][q.next.data-1]):
                                                    Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                else:
                                                    Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                
                                sum += 1
                                break
                            q = q.next
                        p = p.next
                    a = 1
                    Previous = k
                    while(TLA[i-a][Previous].Previous_In > -1):
                        p = TLA[i][j].head
                        while(p != None):
                            q = TLA[i-a][Previous].other
                            while(q != None):
                                if(p.data == q.data):
                                    if(p == TLA[i][j].head):
                                        if(p.next == None):
                                            if(q == TLA[i-a][Previous].other):
                                                if(q.next == None):
                                                    if(TD[i-a-1][q.data-1] < TLA[i][j].Dead_Head):
                                                        Distance[j][k] -= TLA[i][j].Dead_Head
                                                    else:
                                                        Distance[j][k] -= TD[i-a-1][q.data-1]
                                                else:
                                                    if(TD[i-a-1][q.next.data-1]+TLA[i][j].Dead_Head < TD[i-a-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                        Distance[j][k] += (TD[i-a-1][q.next.data-1]-TD[i-a-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                    else:
                                                        Distance[j][k] -= TLA[i][j].Dead_Head
                                            else:
                                                if(q.next == None):
                                                    if(Dead[q.prev.data-1][q.data-1] < TLA[i][j].Dead_Head):
                                                        Distance[j][k] -= TLA[i][j].Dead_Head
                                                    else:
                                                        Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                                else:
                                                    if(Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1] < Dead[q.prev.data-1][q.next.data-1]+TLA[i][j].Dead_Head):
                                                        Distance[j][k] -= TLA[i][j].Dead_Head
                                                    else:
                                                        Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                        else:
                                            if(q == TLA[i-a][Previous].other):
                                                if(q.next == None):
                                                    if(TD[i][p.next.data-1]+TD[i-a-1][q.data-1] < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                        Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                    else:
                                                        Distance[j][k] -= TD[i-a-1][q.data-1]
                                                else:
                                                    if(TD[i-a-1][q.next.data-1] < TD[i][p.next.data-1]):
                                                        Distance[j][k] += (TD[i-a-1][q.next.data-1]-TD[i-a-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                    else:
                                                        Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                            else:
                                                if(q.next == None):
                                                    if(TD[i][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                        Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                    else:
                                                        Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                                else:
                                                    if(Dead[q.prev.data-1][q.next.data-1] < TD[i][p.next.data-1]):
                                                        Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data]-Dead[q.data-1][q.next.data-1])
                                                    else:
                                                        Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                    else:
                                        if(p.next == None):
                                            if(q == TLA[i-a][Previous].other):
                                                if(q.next == None):
                                                    if(Dead[p.prev.data-1][p.data-1] < TD[i-a-1][q.data-1]):
                                                        Distance[j][k] -= TD[i-a-1][q.data-1]
                                                    else:
                                                        Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                                else:
                                                    if(TD[i-a-1][q.data-1]+Dead[q.data-1][q.next.data-1] < Dead[p.prev.data-1][p.data-1]+TD[i-a-1][q.next.data-1]):
                                                        Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                                    else:
                                                        Distance[j][k] += (TD[i-a-1][q.next.data-1]-TD[i-a-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                            else:
                                                if(q.next == None):
                                                    if(Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]):
                                                        Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                                    else:
                                                        Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                                else:
                                                    if(Dead[p.prev.data-1][p.data-1]+Dead[q.prev.data-1][q.next.data-1] < Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                        Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                    else:
                                                        Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                        else:
                                            if(q == TLA[i-a][Previous].other):
                                                if(q.next == None):
                                                    if(Dead[p.prev.data-1][p.next.data-1] + TD[i-a-1][q.data-1] < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                        Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                    else:
                                                        Distance[j][k] -= TD[i-a-1][q.data-1]
                                                else:
                                                    if(TD[i-a-1][q.next.data-1] < Dead[p.prev.data-1][p.next.data-1]):
                                                        Distance[j][k] += (TD[i-a-1][q.next.data-1]-TD[i-a-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                    else:
                                                        Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                            else:
                                                if(q.next == None):
                                                    if(Dead[p.prev.data-1][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                        Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                    else:
                                                        Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                                else:
                                                    if(Dead[p.prev.data-1][p.next.data-1] < Dead[q.prev.data-1][q.next.data-1]):
                                                        Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                    else:
                                                        Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                    sum += 1
                                    break
                                q = q.next
                            p = p.next
                        Previous = TLA[i-a][Previous].Previous_In
                        a += 1
                    l = MAX(x, TLA[i][j].No_of_Loads+y-sum)
                    if(TLA[i][j].No_of_Loads+y-sum == x):
                        if(Dead_Head > Distance[j][k]):
                            Previous_In = k
                            Dead_Head = Distance[j][k]
                    elif(TLA[i][j].No_of_Loads+y-sum > x):
                        Previous_In = k
                        Dead_Head = Distance[j][k]
                    x = l
            if(sum > 0):
                p = TLA[i][j].head
                while(p != None):
                    q = TLA[i-1][Previous_In].head
                    while(q != None):
                        if(p.data == q.data):
                            if(p == TLA[i][j].head):
                                if(p.next == None):
                                    if(q == TLA[i-1][Previous_In].head):
                                        if(q.next == None):
                                            if(TLA[i][j].Dead_Head > TD[i-1][q.data-1]):
                                                TLA[i][j].head = None
                                        else:
                                            if(TD[i-1][q.data-1]+Dead[q.data-1][q.next.data-1] < TLA[i][j].Dead_Head+TD[i-1][q.next.data-1]):
                                                TLA[i][j].head = None
                                    else:
                                        if(q.next == None):
                                            if(Dead[q.prev.data-1][q.data-1] < TLA[i][j].Dead_Head):
                                                TLA[i][j].head = None
                                        else:
                                            if(TLA[i][j].Dead_Head+Dead[q.prev.data-1][q.next.data-1] > Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                TLA[i][j].head = None
                                else:
                                    if(q == TLA[i-1][Previous_In].head):
                                        if(q.next == None):
                                            if(TD[i][p.next.data-1] + TD[i-1][q.data-1] < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                TLA[i][j].head = p.next
                                                p.next.prev = None
                                                p.next = None
                                                p.prev = None
                                        else:
                                            if(TD[i-1][q.next.data-1] > TD[i][p.next.data-1]):
                                                TLA[i][j].head = p.next
                                                p.next.prev = None
                                                p.prev = None
                                                p.next = None
                                    else:
                                        if(q.next == None):
                                            if(TD[i][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                TLA[i][j].head = p.next
                                                p.next.prev = None
                                                p.prev = None
                                                p.next = None
                                        else:
                                            if(TD[i][p.next.data-1] < Dead[q.prev.data-1][q.next.data-1]):
                                                TLA[i][j].head = p.next
                                                p.next.prev = None
                                                p.prev = None
                                                p.next = None
                            else:
                                if(p.next == None):
                                    if(q == TLA[i-1][Previous_In].head):
                                        if(q.next == None):
                                            if(Dead[p.prev.data-1][p.data-1] > TD[i-1][q.data-1]):
                                                p.prev.next = None
                                                p.prev = None
                                        else:
                                            if(TD[i-1][q.data-1]+Dead[q.data-1][q.next.data-1] < Dead[p.prev.data-1][p.data-1]+TD[i-1][q.next.data-1]):
                                                p.prev.next = None
                                                p.prev = None
                                    else:
                                        if(q.next == None):
                                            if(Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]):
                                                p.prev.next = None
                                                p.prev = None
                                        else:
                                            if(Dead[p.prev.data-1][p.data-1]+Dead[q.prev.data-1][q.next.data-1] > Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                p.prev.next = None
                                                p.prev = None
                                else:
                                    if(q == TLA[i-1][Previous_In].head):
                                        if(q.next == None):
                                            if(Dead[p.prev.data-1][p.next.data-1] + TD[i-1][q.data-1] < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                p.prev.next = p.next
                                                p.next.prev = p.prev
                                                p.next = None
                                                p.prev = None
                                        else:
                                            if(TD[i-1][q.next.data-1] > Dead[p.prev.data-1][p.next.data-1]):
                                                p.prev.next = p.next
                                                p.next.prev = p.prev
                                                p.prev = None
                                                p.next = None
                                    else:
                                        if(q.next == None):
                                            if(Dead[p.prev.data-1][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                p.prev.next = p.next
                                                p.next.prev = p.prev
                                                p.prev = None
                                                p.next = None
                                        else:
                                            if(Dead[p.prev.data-1][p.next.data-1] < Dead[q.prev.data-1][q.next.data-1]):
                                                p.prev.next = p.next
                                                p.next.prev = p.prev
                                                p.prev = None
                                                p.next = None
                            break
                        q = q.next
                    p = p.next
            if(sum > 0):
                a = 1
                Previous = Previous_In
                while(TLA[i-a][Previous].Previous_In > -1):
                    p = TLA[i][j].head
                    while(p != None):
                        q = TLA[i-a][Previous].other
                        while(q != None):
                            if(p.data == q.data):
                                if(p == TLA[i][j].head):
                                    if(p.next == None):
                                        if(q == TLA[i-a][Previous].other):
                                            if(q.next == None):
                                                if(TD[i][p.data-1] > TD[i-a-1][q.data-1]):
                                                    TLA[i][j].head = None
                                                else:
                                                    TLA[i-a][Previous].other = None
                                            else:
                                                if(TD[i-a-1][q.data-1]+Dead[q.data-1][q.next.data-1] < TD[i][p.data-1]+TD[i-a-1][q.next.data-1]):
                                                    TLA[i][j].head = None
                                                else:
                                                    TLA[i-a][Previous].other = q.next
                                                    TLA[i-a][Previous].other.prev = None
                                                    q.next = None
                                        else:
                                            if(q.next == None):
                                                if(Dead[q.prev.data-1][q.data-1] < TD[i][p.data-1]):
                                                    TLA[i][j].head = None
                                                else:
                                                    q.prev.next = None
                                                    q.prev = None
                                            else:
                                                if(TD[i][p.data-1]+Dead[q.prev.data-1][q.next.data-1] > Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                    TLA[i][j].head = None
                                                else:
                                                    q.next.prev = q.prev
                                                    q.prev.next = q.next
                                                    q.next = None
                                                    q.prev = None
                                    else:
                                        if(q == TLA[i-a][Previous].other):
                                            if(q.next == None):
                                                if(TD[i][p.next.data-1] + TD[i-a-1][q.data-1] < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    TLA[i][j].head = p.next
                                                    p.next.prev = None
                                                    p.next = None
                                                    p.prev = None
                                                else:
                                                    TLA[i-a][Previous].other = None
                                            else:
                                                if(TD[i-a-1][q.next.data-1] > TD[i][p.next.data-1]):
                                                    TLA[i][j].head = p.next
                                                    p.next.prev = None
                                                    p.prev = None
                                                    p.next = None
                                                else:
                                                    TLA[i-a][Previous].other = q.next
                                                    TLA[i-a][Previous].other.prev = None
                                                    q.next = None
                                        else:
                                            if(q.next == None):
                                                if(TD[i][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    TLA[i][j].head = p.next
                                                    p.next.prev = None
                                                    p.prev = None
                                                    p.next = None
                                                else:
                                                    q.prev.next = None
                                                    q.prev = None
                                            else:
                                                if(TD[i][p.next.data-1] < Dead[q.prev.data-1][q.next.data-1]):
                                                    TLA[i][j].head = p.next
                                                    p.next.prev = None
                                                    p.prev = None
                                                    p.next = None
                                                else:
                                                    q.next.prev = q.prev
                                                    q.prev.next = q.next
                                                    q.prev = None
                                                    q.next = None
                                else:
                                    if(p.next == None):
                                        if(q == TLA[i-a][Previous].other):
                                            if(q.next == None):
                                                if(Dead[p.prev.data-1][p.data-1] > TD[i-a-1][q.data-1]):
                                                    p.prev.next = None
                                                    p.prev = None
                                                else:
                                                    TLA[i-a][Previous].other = None
                                            else:
                                                if(TD[i-a-1][q.data-1]+Dead[q.data-1][q.next.data-1] < Dead[p.prev.data-1][p.data-1]+TD[i-a-1][q.next.data-1]):
                                                    p.prev.next = None
                                                    p.prev = None
                                                else:
                                                    TLA[i-a][Previous].other = q.next
                                                    TLA[i-a][Previous].other.prev = None
                                                    q.next = None
                                        else:
                                            if(q.next == None):
                                                if(Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]):
                                                    p.prev.next = None
                                                    p.prev = None
                                                else:
                                                    q.prev.next = None
                                                    q.prev = None
                                            else:
                                                if(Dead[p.prev.data-1][p.data-1]+Dead[q.prev.data-1][q.next.data-1] > Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                    p.prev.next = None
                                                    p.prev = None
                                                else:
                                                    q.next.prev = q.prev
                                                    q.prev.next = q.next
                                                    q.prev = None
                                                    q.next = None
                                    else:
                                        if(q == TLA[i-a][Previous].other):
                                            if(q.next == None):
                                                if(Dead[p.prev.data-1][p.next.data-1] + TD[i-a-1][q.data-1] < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    p.prev.next = p.next
                                                    p.next.prev = p.prev
                                                    p.next = None
                                                    p.prev = None
                                                else:
                                                    TLA[i-a][Previous].other = None
                                            else:
                                                if(TD[i-a-1][q.next.data-1] > Dead[p.prev.data-1][p.next.data-1]):
                                                    p.prev.next = p.next
                                                    p.next.prev = p.prev
                                                    p.prev = None
                                                    p.next = None
                                                else:
                                                    TLA[i-a][Previous].other = q.next
                                                    TLA[i-a][Previous].other.prev = None
                                                    q.next = None
                                        else:
                                            if(q.next == None):
                                                if(Dead[p.prev.data-1][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    p.prev.next = p.next
                                                    p.next.prev = p.prev
                                                    p.prev = None
                                                    p.next = None
                                                else:
                                                    q.prev.next = None
                                                    q.prev = None
                                            else:
                                                if(Dead[p.prev.data-1][p.next.data-1] < Dead[q.prev.data-1][q.next.data-1]):
                                                    p.prev.next = p.next
                                                    p.next.prev = p.prev
                                                    p.prev = None
                                                    p.next = None
                                                else:
                                                    q.next.prev = q.prev
                                                    q.prev.next = q.next
                                                    q.prev = None
                                                    q.next = None
                                break
                            q = q.next
                        p = p.next
                    Previous = TLA[i-a][Previous].Previous_In
                    a += 1
            p = TLA[i-1][Previous_In].head
            while(p != None):
                q = TLA[i][j].head
                o = 1
                while(q != None):
                    if(p.data == q.data):
                        o = 0
                        break
                    q = q.next
                if(o == 1):
                    if(TLA[i][j].other.data == 0):
                        TLA[i][j].other = Node(p.data)
                        TLA[i][j].tother = TLA[i][j].other
                    else:
                        TLA[i][j].tother.next = Node(p.data)
                        TLA[i][j].tother.next.prev = TLA[i][j].tother
                        TLA[i][j].tother = TLA[i][j].tother.next
                p = p.next
            TLA[i][j].No_of_Loads = x
            TLA[i][j].Dead_Head = Dead_Head
            TLA[i][j].Previous_In = Previous_In
        print(TLA[i][j].Preference, TLA[i][j].No_of_Loads, TLA[i][j].Dead_Head, TLA[i][j].Previous_In)


                            
                                
                        
                        
                                
                                
                        
                            
                    
                

In [ ]:
# s = TLA[1][7].other
# while(s != None):
#     print(s.data)
#     s = s.next

In [ ]:
In_dex = 0
x = TLA[len(T)-1][j].No_of_Loads
dead = TLA[len(T)-1][j].Dead_Head
for j in range(len(L)):
    l = MAX(x, TLA[len(T)-1][j].No_of_Loads)
    if(TLA[len(T)-1][j].No_of_Loads == x):
        if(TLA[len(T)-1][j].Dead_Head < dead):
            In_dex = j
            dead = TLA[len(T)-1][j].Dead_Head
    elif(TLA[len(T)-1][j].No_of_Loads > x):
        In_dex = j
        dead = TLA[len(T)-1][j].Dead_Head
    x = l
s = TLA[len(T)-1][In_dex].head
print("Truck:", len(T))
while(s != None):
    print(s.data)
    s = s.next
print("---------____________---------")
i = len(T)-1
Previous = In_dex
while(TLA[i][Previous].Previous_In > -1):
    p = TLA[i][Previous].other
    print("Truck:", i)
    while(p != None):
        print(p.data)
        p = p.next
    print("---------____________---------")
    Previous = TLA[i][Previous].Previous_In
    i -= 1
print("Total Dead_Head:", TLA[len(T)-1][In_dex].Dead_Head)
print("Total No_of_Loads", TLA[len(T)-1][In_dex].No_of_Loads)

In [ ]:
for ind, l in enumerate(L):
    if ind + 1 in [4, 13, 14]:
        print("{}: EarlyPickupApptTime {}, LatestPickupApptTime {}, \n\tDeliveryApptTime {}, latitude {}, longitude {}, dlatitude {}, dlongitude {}".format(
                            ind + 1, l.pickup.EarlyAppointmentTime, l.pickup.LatestAppointmentTime, 
                      l.delivery.EarlyAppointmentTime, l.pickup.Latitude, l.pickup.Longitude, l.delivery.Latitude, l.delivery.Longitude))
    